In [1]:
import cv2
import numpy as np

In [3]:
PATH = 'test_video.avi'

# Get a VideoCapture object from video and store it in vs
vc = cv2.VideoCapture(PATH)
# Read first frame
ret, first_frame = vc.read()
# Scale and resize image
resize_dim = 600
max_dim = max(first_frame.shape)
scale = resize_dim/max_dim

#first_frame = cv2.resize(first_frame, None, fx=scale, fy=scale)

# Convert to gray scale 
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

# Create mask
mask = np.zeros_like(first_frame)

# Sets image saturation to maximum
mask[:,:,1] = 255

fshape = prev_gray.shape
fheight = fshape[0]
fwidth = fshape[1]

fps = vc.get(cv2.CAP_PROP_FPS)


fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('video.mp4',fourcc,fps,(fwidth, fheight))

while(vc.isOpened()):
    # Read a frame from video
    ret, frame = vc.read()
    
    if frame is None:
        print('video ended')
        break
    
    gray = frame
    # Convert new frame format`s to gray scale and resize gray frame obtained
    #gray = cv2.resize(frame, None, fx=scale, fy=scale)
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)


    # Calculate dense optical flow by Farneback method
    # https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, pyr_scale = 0.5, levels = 5, winsize = 15, iterations = 5, poly_n = 3, poly_sigma = 1.1, flags = 0)
    # Compute the magnitude and angle of the 2D vectors
    magnitude, angle = cv2.cartToPolar(flow[:,:,0], flow[:,:,1])
    # Set image hue according to the optical flow direction
    mask[:,:,0] = angle * 180 / np.pi / 2
    # Set image value according to the optical flow magnitude (normalized)
    mask[:,:,2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    # Convert HSV to RGB (BGR) color representation
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    
    # Resize frame size to match dimensions
    #frame = cv2.resize(frame, None, fx=scale, fy=scale)
    
    # Open a new window and displays the output frame
    dense_flow = cv2.addWeighted(frame, 1,rgb, 2, 0)
    cv2.imshow("Dense optical flow", dense_flow)
    out.write(dense_flow)
    # Update previous frame
    prev_gray = gray
    # Frame are read by intervals of 1 millisecond. The programs breaks out of the while loop when the user presses the 'q' key
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
# The following frees up resources and closes all windows
vc.release()
out.release() 
cv2.destroyAllWindows()

video ended
